In [6]:
import numpy as np
np.random.seed(1000)
import cv2
import os
from PIL import Image
import keras
os.environ["KERAS_BACKEND"]='tensorflow'

In [11]:
image_directory = '/Users/pushkaraggarwal/Downloads/cell_images/cell_images/'
SIZE = 64
dataset = []
label = []

parasitized_images = os.listdir(image_directory + 'Parasitized/')
for i,image_name in enumerate(parasitized_images):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Parasitized/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE,SIZE))
        dataset.append(np.array(image))
        label.append(0)
uninfected_images = os.listdir(image_directory + 'Uninfected/')
for i,image_name in enumerate(uninfected_images):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Uninfected/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE,SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [12]:
INPUT_SHAPE = (SIZE,SIZE,3)
inp = keras.layers.Input(shape=INPUT_SHAPE)

conv1 = keras.layers.Conv2D(32, kernel_size=(3,3),
                         activation='relu',padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
norm1 = keras.layers.BatchNormalization(axis=-1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)

conv2 = keras.layers.Conv2D(32,kernel_size=(3,3),
                         activation='relu',padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2,2))(conv2)
norm2 = keras.layers.BatchNormalization(axis=-1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)

hidden1 = keras.layers.Dense(512,activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)
hidden2 = keras.layers.Dense(512,activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='relu')(drop4)

model=keras.Model(inputs = inp, outputs = out)
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 32)        9248

In [13]:
import sklearn
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

X_train, X_test, y_train ,y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size=0.20, random_state=0)

history = model.fit(np.array(X_train), y_train, batch_size = 64, verbose=1, epochs=25, validation_split=0.1, shuffle=False)

print("Test_Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test),np.array(y_test))[1]*100))

import matplotlib.pyplot as plt
f, (ax1, ax2) = plt.subplots(1,2,figsize=(16,8))
t= f.suptitle('CNN Performance',fontsize=10)
f.subplots_adjust(top=0.85, wspace=0.3)

max_epoch = len(history.history['accuracy'])+1
epoch_list = list(range(1,max_epoch))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['accuracy'], label='Validation Accuracy')
axl.set_xticks(np.arange(1, max_epoch, 5))
axl.set_ylabel('Accuracy Value')
axl.set_xlabel('Epoch')
axl.set_title('Accuracy')
l1=axl.legend(loc='best')

ax2.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax2.plot(epoch_list, history.history['accuracy'], label='Validation Accuracy')
ax2.set_xticks(np.arange(1, max_epoch, 5))
ax2.set_ylabel('Accuracy Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Accuracy')
l2=ax2.legend(loc='best')

model.save('malaria_cnn.h5')

Epoch 1/25
311/311 [==============================] - 60s 187ms/step - loss: nan - accuracy: 0.5033 - val_loss: nan - val_accuracy: 0.4948
Epoch 2/25
 63/311 [=====>........................] - ETA: 44s - loss: nan - accuracy: 0.5069

KeyboardInterrupt: 

In [4]:
import pandas as pd
pd.Series([1,2])

0    1
1    2
dtype: int64